In [2]:
import os
import pandas as pd
log_path= r'C:\Users\gijyutukaihatsu\HIK_Vision_MotionDetect_Sumup\20220530194236logBack.txt'
with open(log_path,'r', encoding='shift_jis',errors='ignore') as f:
    logtxt = pd.DataFrame(f.readlines())
    print('txtファイル読み込み完了')

----------------------------
258454    2021-12-9 4:54:11
----------------------------
メインリスト: アラーム録画
サブリスト: 動体検知開始
ローカルユーザー: N/A
ホストIPアドレス: N/A
パラメータタイプ: N/A
カメラ No.: D1

In [3]:
# r = logtxt.head(n=3000)
r = logtxt
r = r.astype('str')
from datetime import datetime

result = pd.DataFrame(columns=['Date','CameraName','num']).\
         astype({'Date': datetime, 'CameraName' : str, 'num' : int})

print('テキストファイルのDataFrameへの変換完了')

#重複の無いCameraNameのデータフレームを取得する

#子要素として、カメラ毎に時間別に分けられた(resample('H').sum())

#行で分割してリストにする
#サブリスト: 動体検知開始で

num_rows = len(r)

for i, line in r.iterrows():
    if "動体検知開始" in str(line) and i > 4:
      st = f'見つけた {i}行目　{r.iloc[i-3]} {r.iloc[i+4]}'
      # print(st)
      result = result.append({'Date': datetime.strptime(str(r.iloc[i-3].values[0].strip()[10:]), '%Y-%m-%d %H:%M:%S'),'CameraName' : r.iloc[i+4].values[0].replace('カメラ No.: ','').strip(), 'num' : 1} , ignore_index=True)
      if i % 100 == 0:
        print(f'動体検知検索中 {num_rows}行中　{i}行目処理中　')

#カメラ毎のヒストリーデータを入れるカメラリストの準備
df_cameralists = pd.DataFrame(result.loc[:]["CameraName"].drop_duplicates())
df_cameralists['historical_data'] = ''
df_cameralists = df_cameralists.set_index('CameraName')

#カメラ毎にヒストリーデータを入れていく
print('ヒストリーデータの計算開始')
for camera_name, historicaldata in df_cameralists.itertuples():
    # result からカメラ名で抜き出し、Date列で時間毎に集計する
    df_cameralists.at[camera_name,'historical_data'] = result[result['CameraName']==camera_name].set_index('Date').resample('H').sum()
    print(f'{camera_name}の計算完了')

#カメラ毎にCSV出力していく
for camera_name , df_historical_data in df_cameralists.itertuples():
    df_historical_data.to_csv(f'{camera_name}.csv')
    print(f'{camera_name}のcsv出力完了')

KeyboardInterrupt: 

0.0038835975519509814